In [37]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [38]:
weather = pd.read_csv('weather_df.csv')
weather.head()

,city,clouds,country,date,humid,lat,lng,max temperature,wind speed,hemi
0,jamestown,65,AU,1570464674,95,-33.21,138.60,43.97,9.93,southern
1,carballo,90,ES,1570465089,64,43.21,-8.69,75.99,3.36,northern
2,rocha,0,UY,1570464900,64,-34.48,-54.34,67.55,13.42,southern
3,saldanha,63,PT,1570465089,20,41.42,-6.55,80.33,2.28,northern
4,impfondo,100,CD,1570465090,95,1.62,18.06,74.93,1.12,northern


In [39]:
locations = weather[["lat", "lng"]]
humid = weather["humid"]

In [40]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humid,
                                 dissipating=False, max_intensity=120,
                                 point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [74]:
vacation = weather[(weather['humid'] <= 20) & (weather['max temperature'] <= 85) & (weather['clouds'] <= 70)]

In [75]:
vacation

,city,clouds,country,date,humid,lat,lng,max temperature,wind speed,hemi
3,saldanha,63,PT,1570465089,20,41.42,-6.55,80.33,2.28,northern
117,general pico,54,AR,1570464821,20,-35.66,-63.75,69.17,15.61,southern
160,san juan,53,AR,1570465113,16,-31.54,-68.53,65.93,9.69,southern
362,englewood,1,US,1570465046,16,39.65,-104.99,60.01,6.93,northern
367,general roca,0,AR,1570465147,17,-39.03,-67.58,66.20,17.22,southern
482,dawlatabad,0,AF,1570465168,13,36.41,64.91,76.19,10.96,northern
531,cheyenne,1,US,1570465097,11,41.14,-104.82,59.00,12.75,northern
543,ovalle,0,CL,1570465178,11,-30.60,-71.20,81.05,7.17,southern


In [86]:
hotel_pd = pd.DataFrame()
hotel_pd["Hotel Name"] = ""
hotel_pd["Hotel Address"] = ""
hotel_pd["lat"] = ""
hotel_pd["lng"] = ""

hotel_pd

,Hotel Name,Hotel Address,lat,lng


In [99]:
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key}

for index, row in vacation.iterrows():
    lat = row["lat"]
    lng = row["lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    print(json.dumps(name_address, indent=4, sort_keys=True))
    
    try:
        hotel_pd.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_pd.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        hotel_pd.loc[index, "lat"] = name_address["results"][0]["geometry"]["location"]["lat"]
        hotel_pd.loc[index, "lng"] = name_address["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -35.6592484,
                    "lng": -63.7515495
                },
                "viewport": {
                    "northeast": {
                        "lat": -35.65794937010727,
                        "lng": -63.75026097010728
                    },
                    "southwest": {
                        "lat": -35.66064902989272,
                        "lng": -63.75296062989273
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "056ec21e330b8d7924a4d2f039e9b292433b51c8",
            "name": "Euskadi Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
   

In [100]:
hotel_pd

,Hotel Name,Hotel Address,lat,lng
117,Euskadi Hotel,"Calle 22 776, Gral. Pico",-35.6592,-63.7515
160,Hotel América,"9 de Julio Este 1052, San Juan",-31.5416,-68.5107
362,Hyatt Place Denver Cherry Creek,"4150 E Mississippi Ave, Glendale",39.6956,-104.939
367,Hotel Huemul,"Gral. Manuel Belgrano 1479, FEE",-39.0292,-67.5716
531,The Historic Plains Hotel,"1600 Central Ave, Cheyenne",41.1333,-104.815
543,Hotel Limari,"Camino a Sotaqui km 5, Ruta D55, Sector Viñita...",-30.5961,-71.1524


In [132]:
location = hotel_pd[["lat", "lng"]]

In [133]:
markers_layer = gmaps.marker_layer(
        location)

fig = gmaps.figure()
fig.add_layer(markers_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [134]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers_layer)

fig

Figure(layout=FigureLayout(height='420px'))